# 迁移学习
实际中，基本没有人会从零开始（随机初始化）训练一个完整的卷积网络，因为相对于网络，很难得到一个足够大的数据集[网络很深, 需要足够大数据集]。通常的做法是在一个很大的数据集上进行预训练得到卷积网络ConvNet, 然后将这个ConvNet的参数作为目标任务的初始化参数或者固定这些参数。

### 使用场景：

 场景1：微调Convnet

 场景2：将Convnet看成固定的特征提取器


In [ ]:
# from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms    # models：加载预训练模型
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

## 1 加载数据

In [ ]:
'''这里未定义成函数，可直接使用'''

# 数据变换方式(字典)——训练集数据扩充和归一化；在验证集上仅需要归一化
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), #随机裁剪一个area然后再resize
        transforms.RandomHorizontalFlip(), #随机水平翻转
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/hymenoptera_data'
# 创建数据集机及其对应迭代器（字典类型，包含测试集和验证集）
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



## 2 训练模型

In [ ]:
'''调整学习速率，保存最好的模型（通过来自 torch.optim.lr_scheduler的学习速率调整类的对象scheduler）'''

def modelTrain(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # 每个epoch都有一个训练和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # 迭代数据.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 零参数梯度
                optimizer.zero_grad()

                # 前向
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 后向+仅在训练阶段进行优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 统计
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # 深度复制mo
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # 加载最佳模型权重
    model.load_state_dict(best_model_wts)
    
    return model

## 3. 1 场景1：微调ConvNet
使用预训练的网络(如在imagenet 1000上训练而来的网络)来初始化自己的网络，而不是随机初始化。其他的训练步骤不变；重置最终完全连接的图层。

In [ ]:
# 加载预训练模型
model_ft = torchvision.models.resnet18(pretrained=True)

# 重置最后的全连接层
num_filters = model_ft.fc.in_features    # 最后一层的特征数
model_ft.fc = nn.Linear(num_filters, 2)  # 将最后一层变为输出为2的全连接层

# GPU加速
model_ft = model_ft.to(device)

#  定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# 学习速率调整类，每7个epochs衰减lr（乘以gamma）
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


## 训练模型
model_ft = modelTrain(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)



## 3.2 场景2：ConvNet作为固定特征提取器
固定ConvNet除了最后的全连接层外的其他所有层，最后的全连接层被替换成一个新的随机初始化的层，只有这个新的层会被训练

In [ ]:
# 加载预训练模型
model_conv = torchvision.models.resnet18(pretrained=True)
# 冻结所有网络层-----------
for param in model_conv.parameter():
    parameter.requires_grad = False
    
# 重置最后的全连接层（新添加的层requires_grad默认为True）
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

# GPU加速
model_conv = model_conv.to(device)

#  定义损失函数和优化器（优化器只定义最后一层）---------
criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)


## 训练模型
model_conv = modelTrain(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)
